# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [26]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [27]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [28]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [29]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster and establish connection

In [30]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [31]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [32]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### QUESTION 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### Create table

List of columns should consist all information that SELECT query would return plus columns needed for PRIMARY KEY.

The columns which query should return: `artist, song, length`. 

As observed the data in `event_datafile_new.csv` file, we found `PRIMARY KEY` should be composite of 2 columns: `sessionId` and `itemInSession`,
because together they identify a row unique. 
As request, the query also just use those 2 columns to filtering data so we don't need any additional column for `PRIMARY KEY`.

In [33]:
create_table1 = """
    CREATE TABLE IF NOT EXISTS songs_by_session (
        sessionId int, itemInSession int, artist text, song text, length decimal,
        PRIMARY KEY (sessionId, itemInSession)
    )
"""

try:
    session.execute(create_table1)
except Exception as e:
    print(e)

##### Insert data into table

Following steps will be performed:
1. read data from event_datafile_new.csv file into a pandas dataframe.
2. build another dataframe consisting value of columns which match 1-1 to table columns.
3. write INSERT query.
4. loop through each datarow of dataframe in step 2:
  * pass data of each row into INSERT query of step 3.
  * exexute INSERT query to add record into table.

In [34]:
import decimal
file = 'event_datafile_new.csv'

df = pd.read_csv(file)

df_insert1 = df[['sessionId', 'itemInSession', 'artist', 'song', 'length']]

insert1 = "INSERT INTO songs_by_session(sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s) "

for i, row in df_insert1.iterrows():
    session.execute(insert1, list(row))

##### Run SELECT query to verify data

In [35]:
query1 = " select artist, song, length from songs_by_session where sessionId = 338 and itemInSession = 4 "

try:
    rows = session.execute(query1)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


#### QUESTION 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### Create table

List of columns should consist all information that SELECT query would return plus columns needed for PRIMARY KEY.

The columns which query should return: `artist`, `song`, `firstName` and `lastName`.

Since the query will filter data by `userId` and `sessionId`, so we also need to include them into PRIMARY KEY, and together they make PARTITION KEY.

Beside of that, we also need `itemInSession` column since the records need to to sorted by this column, so it will be CLUSTERING COLUMNS and we need to have `WITH CLUSTERING ORDER BY (itemInSession DESC)` statement when create table.

To summarize, we will have a COMPROUND PRIMARY KEY for this table which will have a COMPOSITE PARTITION KEY with 2 columns `userId`, `sessionId`, and another 'itemInSession' as CLUSTERING COLUMN. This COMPROUND PRIMARY KEY also ensures a row unique among data.

In [36]:
create_table2 = """
    CREATE TABLE IF NOT EXISTS songs_by_users (
        userId int, sessionId int, itemInSession int, artist text,    
        song text, firstName text, lastName text,    
        PRIMARY KEY ((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession DESC);
"""

try:
    session.execute(create_table2)
except Exception as e:
    print(e)

##### Insert data into table

Steps will be perform are same with what we did with QUESTION 1. I will not read .csv file again since we already did in QUESTION 1, we will just select data from colums we need here, then iterate through each row to passing values into INSERT query. 

In [38]:
df_insert2 = df[['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName']]

insert2 = "INSERT INTO songs_by_users(userId, sessionId, itemInSession, artist, song, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s) "
 
for i, row in df_insert2.iterrows():
    session.execute(insert2, list(row))

##### Run SELECT query to verify data

In [39]:
query2 = " select artist, song, firstName, lastName from songs_by_users where userid = 10 and sessionId = 182 "
try:
    rows = session.execute(query2)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


#### QUESTION 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### Create table

List of columns should consist all information that SELECT query would return plus columns needed for PRIMARY KEY.

The columns which query should return: `firstName` and `lastName`. 

Since the query will filter data by `song`, so we also need it in PRIMARY KEY.

But only `song` column for PRIMARY KEY is not enough since a specific song would be listened serveral times by different users, so I would include `userId` column into PRIMARY KEY as well.

In this case, `song` will be PARTITION KEY since it is the unique condition of WHERE clause, and `userId` would be CLUSTERING COLUMN.

In [40]:
create_table3 = """
    CREATE TABLE IF NOT EXISTS users_listened (
        song text, userId int, firstName text, lastName text,
        PRIMARY KEY ((song), userId))
"""
try:
    session.execute(create_table3)
except Exception as e:
    print(e)

##### Insert data into table

Similar to QUESTION 2, I will not read .csv file again since we already did before, we will just select data from colums we need here, then iterate through each row to passing values into INSERT query.

In [41]:
df_insert3 = df[['song', 'userId', 'firstName', 'lastName']]

insert3 = "INSERT INTO users_listened(song, userId, firstName, lastName) VALUES (%s, %s, %s, %s) "

for i, row in df_insert3.iterrows():
    session.execute(insert3, list(row))

##### Run SELECT query to verify data

In [42]:
query3 = " select firstName, lastName from users_listened where song = 'All Hands Against His Own' "
try:
    rows = session.execute(query3)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [43]:
drop_queries = ["drop table songs_by_session", "drop table songs_by_users", "drop table users_listened"]

try:
    for drop in drop_queries:
        session.execute(drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [44]:
session.shutdown()
cluster.shutdown()